# Tokenization

## Installation


Install `nltk` and download its data.

```bash
pip install nltk
python -m nltk.downloader punkt
```

Then it will download the data in the home directory:
```text
[nltk_data] Downloading package punkt to
[nltk_data]     /GPFS/rhome/xiyuanyang/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
```


## Quick Startup

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

# step1: download some data
nltk.download("punkt_tab")

text = "This is an example sentence, with punctuation!"
words = word_tokenize(text)
print(words)

# for some chinese demo
text_chn = "我是上海交通大学的一名学生,你觉的我怎么样，I am very happy to see you!"
# nltk is quite foolish...
words_chn = word_tokenize(text_chn)
print(words_chn)

# for chinses, you can use jieba
import jieba


text = "我爱北京天安门，天安门上太阳升。"
# 精确模式
seg_list_precise = jieba.cut(text, cut_all=False)
print("精确模式:", " ".join(seg_list_precise))

# 全模式
seg_list_all = jieba.cut(text, cut_all=True)
print("全模式:", " ".join(seg_list_all))

# 搜索引擎模式
seg_list_search = jieba.cut_for_search(text)
print("搜索引擎模式:", " ".join(seg_list_search))

## Encoding and Decoding

We will use `spacy` for more advanced NLP usage.

```bash
pip install spacy
python -m spacy download en_core_web_sm
```

```text
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Collecting en-core-web-sm==3.8.0
  Downloading https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 728.1 kB/s eta 0:00:00
Installing collected packages: en-core-web-sm
Successfully installed en-core-web-sm-3.8.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
```

In [ ]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

text = "This is an example sentence, with punctuation!"
doc = nlp(text)

words = [token.text for token in doc]
print(words)

for token in doc:
    print(f"{token.text:<15} {token.pos_:<10} {token.is_punct}")

In [ ]:
# for encoding and decoding
import spacy

nlp = spacy.load("en_core_web_sm")
text = "Hello world, I am Xiyuan Yang, a freshman in Shanghai JiaoTong University."
doc = nlp(text)

# Get the tokenized words (tokens)
tokens = [token.text for token in doc]
print("SpaCy Tokens:", tokens)

# --- Encode (Get Token Hashes or IDs) ---
# spaCy internally uses hash values to represent strings, and these hashes map to integer IDs.
# token.norm is the normalized hash value (lowercase, etc.)
# token.orth is the hash value of the original string (case-sensitive).

token_ids = [nlp.vocab.strings.as_int(token.text) for token in doc]
print("Token IDs (encoded):", token_ids)


# --- Decode (Convert IDs back to Tokens) ---
decoded_tokens = [nlp.vocab.strings.as_string(id_val) for id_val in token_ids]
print("Decoded Tokens:", decoded_tokens)

# --- More advanced Vocab usage ---
# The `nlp.vocab.strings` object is a StringStore, which manages all string-to-ID mappings.
print("\nSpaCy Vocab Example:")
print("Current vocab size:", len(nlp.vocab))

# Add a new word to the vocabulary and Get the ID of the new word
new_word = "neural_network"
nlp.vocab.strings.add(new_word)
new_word_id = nlp.vocab.strings.as_int(new_word)
print(f"Added '{new_word}', its ID is: {new_word_id}")

# Decode the ID back to the string
decoded_new_word = nlp.vocab.strings.as_string(new_word_id)
print(f"Decoding ID {new_word_id}: {decoded_new_word}")

# If I change the hash value...
try:
    modified_id = new_word_id + 1
    modified_word = nlp.vocab.strings.as_string(modified_id)
    print(f"After Modification: {modified_word}")
except Exception as e:
    print(f"Error: {e}")

## Advanced Tokenizer

We will use `tiktoken` for advanced tokenizer.

In [ ]:
import tiktoken

# Choose an encoding that matches the models you might be using (e.g., for GPT-4, GPT-3.5-turbo)
encoding = tiktoken.get_encoding("cl100k_base")
text_en = "Hello, world! This is a test sentence for tiktoken."

# --- Encode (Text to Token IDs) ---
# encode() converts the string into a list of integer token IDs
token_ids_en = encoding.encode(text_en)
print(f"Original English Text: '{text_en}'")
print(f"Encoded Token IDs (English): {token_ids_en}")
print(f"Number of tokens (English): {len(token_ids_en)}")

# To see the actual tokens that correspond to the IDs (optional, for understanding)
# This requires decoding each ID individually or using a helper.
# Note: decoding individual IDs might not always yield readable strings if tokens are sub-word units.
decoded_parts_en = [encoding.decode([token_id]) for token_id in token_ids_en]
print(f"Decoded Parts (English, for understanding): {decoded_parts_en}")

# --- Decode (Token IDs to Text) ---
# decode() converts a list of integer token IDs back into a string
decoded_text_en = encoding.decode(token_ids_en)
print(f"Decoded English Text: '{decoded_text_en}'")

decoded_modified = [encoding.decode([token_id + 1]) for token_id in token_ids_en]
print(decoded_modified)

# Check if decoded text matches original (should be True)
print(f"Decoded matches original? {decoded_text_en == text_en}")
print(f"After Modifications? {decoded_modified == text_en}")

In [40]:
import tiktoken

test_strings = [
    "Hello world, My name is Xiyuan Yang",
    "wow, it is so fantastic!",
    "你好，这里是中文，自古逢秋悲寂寥，我言秋日胜春朝",
    "international computational"
]
with open("../../README.md", "r", encoding="utf-8") as file:
    long_string = file.read()
    file.close()
test_strings.append(long_string)
encoding = tiktoken.get_encoding("cl100k_base")

for test_string in test_strings:
    tokens = encoding.encode(test_string)
    num_bytes = len(bytes(test_string, encoding="utf-8"))
    # print(num_bytes)
    num_tokens = len(tokens)
    decoded = [encoding.decode([token]) for token in tokens]
    print(decoded)
    print(num_bytes / num_tokens)

['Hello', ' world', ',', ' My', ' name', ' is', ' X', 'iy', 'uan', ' Yang']
3.5
['wow', ',', ' it', ' is', ' so', ' fantastic', '!']
3.4285714285714284
['你', '好', '，', '这', '里', '是', '中', '文', '，', '自', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '�', '，', '我', '言', '�', '�', '日', '�', '�', '�', '�', '�', '�']
2.057142857142857
['international', ' computational']
13.5
['#', ' Learn', ' L', 'LM', 's', ' from', ' scratch', '\n\n', '>', ' [', '!', 'WARNING', ']\n', '>', ' Still', ' in', ' the', ' process', ' of', ' **', 'ref', 'actoring', '**', '!\n\n', '##', ' Table', ' of', ' Contents', '\n\n', 'See', ' [', 'This', ' Blog', '](', 'https', '://', 'xi', 'yu', 'any', 'ang', '-code', '.github', '.io', '/posts', '/', 'LL', 'M', '-L', 'earning', '-', 'Initial', '/)', ' for', ' more', ' tutorials', '.\n\n\n', '##', ' L', 'LM', ' Learning', ' Materials', '\n\n', '-', ' Courses', ' and', ' Videos', '\n\n', '   ', ' -', ' [', 'Post', ' Training', ' by', ' Deep', 'Learning', ' Ai',